In [1]:
import numpy as np
import matplotlib.pyplot as plt

from bayesian_network import BayesNet
from utils import sample_forward, get_default_bayes_net

# Parameter Learning

In this problem, we will assume that a fixed dependency graph structure between variables is given and learn the parameters (the complete Conditional Probability Distribution Table (CPDT)) from a set of events. Furthermore, we will use log-likelihood to find a model structure that also generalizes to future data.
    
## ML Estimates for Conditional Distributions

<div class="alert alert-warning">
    Implement the <i>maximum_likelihood_estimate</i> function, which computes the Maximum Likelihood Estimate for the parameters of a discrete (conditional) probability distribution $ P(X \mid \mathit{pa}(X) )$, given a data set. (3 points)
</div>

`maximum_likelihood_estimate` takes  three parameters:
- `data` is a NumPy array of shape `(num_samples, num_variables)`.
- `variable_id` is the column index of the variable to estimate the distribution for.
- `parent_ids` is a tuple, containing the column indices of parent variables.
- `alpha` is a non-negative integer, corresponding to pseudocounts in Laplace smoothing.

`maximum_likelihood_estimate` must return one object:
- A Maximum Likelihood Estimate (MLE) of the parameters in form of a `np.ndarray`. The first dimension (index `0`) of the returned array must correspond to variable `variable_id`, the remaining dimensions must be sorted according to `parent_ids`. Altogether, tuple `(variable_id, ) + parent_ids` gives the mapping of dimensions to variables.

Hint:
- Assume that all variables are boolean.
- To count elements in a Numpy array, you simply loop over the data array.
- The smoothing parameter `alpha` is added to the counts of each possible event represented in the CPDT.

In [2]:
def maximum_likelihood_estimate(data: np.ndarray, variable_id: int, parent_ids: tuple=tuple(), alpha: int=0):
    """
    Estimates the conditional probability distribution of a (discrete) variable from data.
    :param data:    data to estimate distribution from
    :param variable_id:  column index corresponding to the variable we estimate the distribution for
    :param parent_ids: column indices of the variables the distribution is conditioned on
    :param alpha: smoothing parameter, pseudocounts
    :returns: estimated conditional probability distribution table
    """
    
    assert type(variable_id) == int
    assert type(parent_ids) == tuple
    
    # mapping of axis to variable_id,
    # e.g. the variable with id variable_ids[i] is on axis i of the CPDT
    variable_ids = (variable_id,) + parent_ids
    
    # create output array:
    # in the instructions stand that we can assume all variables to be boolean:
    shaper = tuple([2]*(1+len(parent_ids)))
    cpdt_counter = np.zeros(shaper)
    
    
    for sample in data:
        
        var_value_id = sample[variable_id]
        count_mask = [var_value_id]
        
        for put_in_id, parent_id in enumerate(parent_ids):
            
            parent_value_id = sample[parent_id]
            count_mask.append(parent_value_id)
            
        cpdt_counter[tuple(count_mask)] += 1 
    
    denominator = np.sum(cpdt_counter, axis=0, keepdims=True) # at axis=0 we have the variable of interest

    cpdt = (cpdt_counter+alpha)/(denominator+alpha*2) # the 2 represents "k", the number of possible 
    # variable values (binary --> 2)

    return cpdt
    

In [3]:
# sanity checks
_A_, _B_, _C_, _D_, _E_ = 0, 1, 2, 3, 4
# get the bayes net from the previous problem
bayes_net = get_default_bayes_net()
np.random.seed(0)
# draw 100 samples
data = sample_forward(bayes_net, 100)

# get exact A form bayes net
expected = bayes_net[_A_].pdt[:,0,0,0,0]
# estimate A from the data
actual = maximum_likelihood_estimate(data, _A_)
# estimate should not be far off
assert np.all(np.isclose(expected, actual, atol=0.05))

# get exact B_A form bayes net
expected = bayes_net[_B_].pdt[:,:,0,0,0].T
# estimate B_A from data
actual = maximum_likelihood_estimate(data, _B_, (_A_,))
# estimate should not be far off
assert np.all(np.isclose(expected, actual, atol=0.05))

# test if alpha correctly added
expected = [0.29166667, 0.70833333]
# estimate A from the data with alpha=10
actual = maximum_likelihood_estimate(data, _A_, alpha=10)
# estimate should not be far off
assert np.all(np.isclose(expected, actual, atol=0.0001))


### The Log-Likelihood Function

<div class="alert alert-warning">
    Implement the <i>log_likelihood</i> function, which computes the log-likelihood $\mathcal{L}(\mathcal{M} : \mathcal{D})$ of a model (BayesNet) relative to a data set. (3 points)
</div>

`log_likelihood` takes two parameters:
- `data` is a NumPy array of shape `(num_samples, num_variables)`.
- `bayes_net` a BayesNet object representing the model $\mathcal{M}$ (containing already estimated CPDTs).

`log_likelihood` must return one object:
- The log-likelihood of the model given the data (i.e., a floating point number (<= 0)).

Hint:
- Recall that iterating over the variables in the BayesNet is super easy: `for variable in bayes_net: ...`.
- The probability distribution of variable $X$ given its parents $\mathit{pa}(X)$, $P(X \mid \mathit{pa}(X))$, can be obtained by passing the random event to the variable, i.e., `variable(data[i])`.
- Use the natural logarithm for your computations, i.e. `np.log`.

In [4]:
def log_likelihood(data: np.ndarray, bayes_net: BayesNet):
    """
    Computes the log-likelihood of a given Bayesian network relative to the data.
    :param data: data to compute the log-likelihood relative to.
    :param bayes_net: Bayesian network model.
    :returns: the log-likelihood of the Bayesian network relative to the data.
    """    

    ll = 0
    
    for sample_ind, sample in enumerate(data):
        
        for variable in bayes_net:
            
            P_X_paX = variable(sample)
            
            ll += np.log(P_X_paX[sample[variable.id]]) # only consider the event which actually happend:
            # sample[variable.id]

    return ll

In [5]:
# sanity checks
# get the bayes net from the previous problem
bayes_net = get_default_bayes_net()
np.random.seed(0)
# draw 100 samples
data = sample_forward(bayes_net, 100)

# expected log-likelihood
expected = -215.9
# actual log-likelihood
actual = log_likelihood(data, bayes_net)

# must be close
assert np.all(np.isclose(expected, actual, atol=0.1))


# remove unused variables
del data
del bayes_net

## Finding a Model for Strokes   

After watching hours of medical dramas on television, a medicine freshman tries to figure out the perfect prediction model for strokes. Some of her computer science colleagues told her about how Bayesian networks can be used for symptom diagnosis, so she decides to model her ideas using this technique. In order to find out the (conditional) probability distributions, she needs to find a lot of training examples.

Let's assume that we magically know the true underlying Bayes model (structure and parameters) and we can generate the data for her:

All variables in this example are boolean (false=0 or true=1).  

<img  style='width:100%;  max-width:400px;' src="img/bn_mod2.svg">

The conditional probability tables are given as follows:

<table style="float: left;margin:5px;"><tr><th>P(A)</th><th>$a_0$<br></th><th>$a_1$</th></tr><tr><td>-</td><td>0.01</td><td>0.99</td></tr></table>

<table style="float: left;margin:5px;"><tr><th>P(H | A)</th><th>$a_0$<br></th><th>$a_1$</th></tr><tr><td>$h_0$</td><td>0.9</td><td>0.8</td></tr><tr><td>$h_1$</td><td>0.1</td><td>0.2</td></tr></table>

<table style="float: left;margin:5px;"><tr><th>P(S | H)</th><th>$h_0$<br></th><th>$h_1$</th></tr><tr><td>$s_0$</td><td>0.9</td><td>0.85</td></tr><tr><td>$s_1$</td><td>0.1</td><td>0.15</td></tr></table>


<table style="float: left;margin:5px;"><tr><th rowspan="2">P(C | A, S)</th><th colspan="2">$a_0$<br></th><th colspan="2">$a_1$</th></tr><tr><td>$s_0$</td><td>$s_1$</td><td>$s_0$</td><td>$s_1$</td></tr><tr><td>$c_0$<br></td><td>0.8</td><td>0.7</td><td>0.85</td><td>0.45</td></tr><tr><td>$c_1$</td><td>0.2</td><td>0.3</td><td>0.15</td><td>0.55</td></tr></table>

<table style="float: left;margin:5px;"><tr><th>P(V | S)</th><th>$s_0$</th><th>$s_1$</th></tr><tr><td>$v_0$</td><td>0.1</td><td>0.2</td></tr><tr><td>$v_1$</td><td>0.9</td><td>0.8</td></tr></table>  

Let's sample 5000 events from this network as the training data, and 5000 samples as the test data for her.

In [6]:
_A_, _H_, _S_, _C_, _V_ = 0, 1, 2, 3, 4
A = np.array([0.01, 0.99])
H_A = np.array([[0.9, 0.8], [0.1, 0.2]])
S_H = np.array([[0.9, 0.85], [0.1, 0.15]])
C_AS = np.array([[[0.8, 0.7], [0.85, 0.45]], [[0.2, 0.3], [0.15, 0.55]]])
V_S = np.array([[0.1, 0.2], [0.9, 0.8]])

# this bayes net represents the true underlying full joint distribution in the medical world 
# of our medicine freshman
true_bayes_net = BayesNet(
    (A, (_A_,)),
    (H_A, (_H_,_A_)),
    (S_H, (_S_,_H_)),
    (C_AS, (_C_,_A_,_S_)),
    (V_S, (_V_,_S_))
)
np.random.seed(0)
train = sample_forward(true_bayes_net, 5000)
test = sample_forward(true_bayes_net, 5000)

<div class="alert alert-warning">
    Based on the sampled training data points, estimate the (conditional) probability tables for the true underlying network structure and compute its log-likelihood w.r.t the training data. (1 point)
</div>
  
  
Store the CPDTs into the provided variables. The dimensions of the CPDT must be sorted according to the naming of the variable, e.g., in C_AS, dimension 0 corresponds to C, dimension 1 to A, and dimension 2 to S.

**Hint**:
- Use the two functions you implemented above (`maximum_likelihood_estimate` and `log_likelihood`)!
- The training data is stored in variable `train`. 
- `_A_, _H_, _S_, _C_, _V_` hold the column indices (= IDs) of the variables. 

In [7]:
_A_, _H_, _S_, _C_, _V_ = 0, 1, 2, 3, 4

A, H_A, S_H, C_AS, V_S = None, None, None, None, None

for variable in true_bayes_net:
    
    P_est = maximum_likelihood_estimate(data=train, variable_id=variable.id, parent_ids=tuple(variable.parents), alpha=0)
    
    if variable.id == 0:
        A = P_est
    elif variable.id == 1:
        H_A = P_est
    elif variable.id == 2:
        S_H = P_est
    elif variable.id == 3:
        C_AS = P_est
    elif variable.id == 4:
        V_S = P_est
    

# begin sanity check
assert np.all(np.isclose(A.sum(axis=0), 1))
assert np.all(np.isclose(H_A.sum(axis=0), 1))
assert np.all(np.isclose(S_H.sum(axis=0), 1))
assert np.all(np.isclose(C_AS.sum(axis=0), 1))
assert np.all(np.isclose(V_S.sum(axis=0), 1))
# end sanity check

bayes_net_1 = BayesNet(
    (A, (_A_,)),
    (H_A, (_H_,_A_)),
    (S_H, (_S_,_H_)),
    (C_AS, (_C_,_A_,_S_)),
    (V_S, (_V_,_S_))
)

tr_log_likelihood_1 = log_likelihood(data=train, bayes_net=bayes_net_1)



In [8]:
# sanity check
assert tr_log_likelihood_1 < -8500
assert tr_log_likelihood_1 > -8800


Back to our medicine freshman: Having no idea of the true underlying network structure, she decides to try out the following very simple model first:
    
<img style='width:100%;  max-width:400px;' src="img/bn_mod1.svg">

<br>

<div class="alert alert-warning">
    Based on the sampled training data points, estimate the (conditional) probability tables for the this model and compute its log-likelihood w.r.t the training data. (1 point)
</div>

Store the CPDTs into the provided variables.

**Hint**:
- Use the two functions you implemented above (`maximum_likelihood_estimate` and `log_likelihood`)!
- The training data is stored in variable `train`. 
- `_A_, _H_, _S_, _C_, _V_` hold the column indices (= IDs) of the variables. 

In [9]:
_A_, _H_, _S_, _C_, _V_ = 0, 1, 2, 3, 4

A, H, S, C, V = None, None, None, None, None

for variable in true_bayes_net:
    
    P_est = maximum_likelihood_estimate(data=train, variable_id=variable.id, parent_ids=tuple(), alpha=0)
    
    if variable.id == 0:
        A = P_est
    elif variable.id == 1:
        H = P_est
    elif variable.id == 2:
        S = P_est
    elif variable.id == 3:
        C = P_est
    elif variable.id == 4:
        V = P_est
    


# begin sanity check
assert np.all(np.isclose(A.sum(axis=0), 1))
assert np.all(np.isclose(H.sum(axis=0), 1))
assert np.all(np.isclose(S.sum(axis=0), 1))
assert np.all(np.isclose(C.sum(axis=0), 1))
assert np.all(np.isclose(V.sum(axis=0), 1))
# end sanity check

bayes_net_2 = BayesNet(
    (A, (_A_,)),
    (H, (_H_,)),
    (S, (_S_,)),
    (C, (_C_,)),
    (V, (_V_,))
)

tr_log_likelihood_2 = log_likelihood(data=train, bayes_net=bayes_net_2)

In [10]:
# sanity check
assert tr_log_likelihood_2 < -8500
assert tr_log_likelihood_2 > -8800



Unhappy with the result, she decides to try out a second, more complex model:

<img  style='width:100%;  max-width:400px;' src="img/bn_mod3.svg">

<div class="alert alert-warning">
    Based on the sampled training data points, estimate the (conditional) probability tables for the this model and compute its log-likelihood w.r.t the training data. (1 point)
</div>

Store the CPDTs into the provided variables. The dimensions of the CPDT must be sorted according to the naming of the variable, e.g., in C_AS, dimension 0 corresponds to C, dimension 1 to A, and dimension 2 to S.

**Hint**:
- Use the two functions you implemented above (`maximum_likelihood_estimate` and `log_likelihood`)!
- The training data is stored in variable `train`. 
- `_A_, _H_, _S_, _C_, _V_` hold the column indices (= IDs) of the variables. 

In [11]:
_A_, _H_, _S_, _C_, _V_ = 0, 1, 2, 3, 4

A, H_A, S_AH, C_AS, V_CS = None, None, None, None, None

for variable in true_bayes_net:
        
    if variable.id == 0:
        variable_parents = []
    elif variable.id == 1:
        variable_parents = [_A_] # list can be converted in tuple then
    elif variable.id == 2:
        variable_parents = [_A_,_H_]
    elif variable.id == 3:
        variable_parents = [_A_, _S_]
    elif variable.id == 4:
        variable_parents = [_C_, _S_] # later in function maximum_likelohood_estimate the variables get sorted
    
    P_est = maximum_likelihood_estimate(data=train, variable_id=variable.id, parent_ids=tuple(variable_parents), alpha=0)
    
    if variable.id == 0:
        A = P_est
    elif variable.id == 1:
        H_A = P_est
    elif variable.id == 2:
        S_AH = P_est
    elif variable.id == 3:
        C_AS = P_est
    elif variable.id == 4:
        V_CS = P_est
    

# begin sanity check
assert np.all(np.isclose(A.sum(axis=0), 1))
assert np.all(np.isclose(H_A.sum(axis=0), 1))
assert np.all(np.isclose(S_AH.sum(axis=0), 1))
assert np.all(np.isclose(C_AS.sum(axis=0), 1))
assert np.all(np.isclose(V_CS.sum(axis=0), 1))
# end sanity check

bayes_net_3 = BayesNet(
    (A, (_A_,)),
    (H_A, (_H_,_A_)),
    (S_AH, (_S_,_A_,_H_)),
    (C_AS, (_C_,_A_,_S_)),
    (V_CS, (_V_,_C_,_S_))
)

tr_log_likelihood_3 =  log_likelihood(data=train, bayes_net=bayes_net_3)

In [12]:
# sanity check
assert tr_log_likelihood_3 < -8500
assert tr_log_likelihood_3 > -8800


### Compare Train Log-Likelihoods

Compare the log-likelihoods w.r.t the training data of Model **M1** (having the true underlying structure) to the two models our medicine freshman came up with (**M2** - no edges, **M3** - complex model).

In [13]:
print('logP(train|M1) = {}'.format(tr_log_likelihood_1))
print('logP(train|M2) = {}'.format(tr_log_likelihood_2))
print('logP(train|M3) = {}'.format(tr_log_likelihood_3))

logP(train|M1) = -8508.579282438383
logP(train|M2) = -8740.033479010004
logP(train|M3) = -8504.427454360399


<div class="alert alert-warning">
    Answer the following question in one sentence! (1 point)
</div>

Even though **M1** has the true underlying network structure (it correctly represents all independencies holding in our world), it doesn't have the highest train log-likelihood. How do you explain this?

The complex model covers all true edge relations and in addition two edges which are connected to the involved nodes by estimating P(X|Y) such that it delivers some kind of related information and also the dataset is only an estimation of the ground truth which can't be perfectly represented.

### Compare Test Log-Likelihoods

Finally, we compute the test log-likelihood of the model **M1** (having the true underlying structure) and the models our medicine freshman created **M2** and **M3**.

In [14]:
te_log_likelihood_1 = log_likelihood(test, bayes_net_1)
te_log_likelihood_2 = log_likelihood(test, bayes_net_2)
te_log_likelihood_3 = log_likelihood(test, bayes_net_3)

print('logP(test|M1) = {}'.format(te_log_likelihood_1))
print('logP(test|M2) = {}'.format(te_log_likelihood_2))
print('logP(test|M3) = {}'.format(te_log_likelihood_3))

logP(test|M1) = -8376.49246068484
logP(test|M2) = -8598.449646741903
logP(test|M3) = -8379.468463963516


<div class="alert alert-warning">
    Answer the following question! Keep your answers short! (1 point)
</div>

What is the difference w.r.t. the log-likelihoods of the training data? Explain the difference!

For case of the training data the model the model was already trained on it. It seems to be overfitting the case as the models explain the test set better than the training set. Visibile becomes that through the higher log-likelihood values by using the test set.

### Laplace Smoothing

<div class="alert alert-warning">
    Answer the following question! Keep your answer short! (1 point)
</div>

Our medicine freshman wants to estimate the (conditional) probability tables for her model **M3** again. However, this time she only has a training set consisting of 100 samples. She runs into the error shown in the output of the code cell below. Explain to her, what the source of the problem is and how she can avoid it by adapting a parameter when calling the function ```maximum_likelihood_estimate```.

By calculating the P(X|Y) estimate with formula (N[X,Y]/N[Y]) it can happen that some values of Y simply don't exist in the dataset such that the denominator becomes 0 and we end up with nan values in the P(X|Y) estimates. Therefore, alpha has to be bigger than zero which is an input of the maximum_likelihood_estimate function such that we have P(X|Y) estimate of form (N[X,Y]+alpha)/(N[Y]+alpha*number_of_possible_X_values). Through that the denominator doesn't become 0 again.

In [15]:
np.random.seed(0)
# we generate a new training set consisting of 100 samples for our medicine freshman
train = sample_forward(true_bayes_net, 100)  

A = maximum_likelihood_estimate(train, _A_)
H_A = maximum_likelihood_estimate(train, _H_, (_A_,))
S_AH = maximum_likelihood_estimate(train, _S_, (_A_, _H_,))
C_AS = maximum_likelihood_estimate(train, _C_, (_A_, _S_))
V_CS = maximum_likelihood_estimate(train, _V_, (_S_,_C_,))

bayes_net_3 = BayesNet(
    (A, (_A_,)),
    (H_A, (_H_,_A_)),
    (S_AH, (_S_,_A_,_H_)),
    (C_AS, (_C_,_A_,_S_)),
    (V_CS, (_V_,_C_,_S_))
)

/tmp/ipykernel_6576/4013883794.py:38: RuntimeWarning: invalid value encountered in true_divide
  cpdt = (cpdt_counter+alpha)/(denominator+alpha*2) # the 2 represents "k", the number of possible


AssertionError: Probabilities on axis 0 have to sum to 1!